<a href="https://colab.research.google.com/github/vkjadon/tf/blob/main/tf203-dogsvscats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import shutil
import glob

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

In [2]:
!mkdir ~/.kaggle 

In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle competitions download -c dogs-vs-cats

 99% 806M/812M [00:03<00:00, 230MB/s]
100% 812M/812M [00:04<00:00, 213MB/s]


In [7]:
!unzip dogs-vs-cats.zip

Archive:  dogs-vs-cats.zip
  inflating: sampleSubmission.csv    
  inflating: test1.zip               
  inflating: train.zip               


In [5]:
!pwd

/content


In [ ]:
!unzip train.zip

In [9]:
!ls

dogs-vs-cats.zip  sample_data		test1.zip  train.zip
kaggle.json	  sampleSubmission.csv	train


In [10]:
filename=glob.glob('train/cat*')
print(len(filename))

12500


In [11]:
if os.path.isdir('valid/dog') is False:
  os.makedirs('valid/dog')
  for c in random.sample(glob.glob('train/dog*'),1000):
    shutil.move(c,'valid/dog')

In [12]:
filename=glob.glob('valid/dog/*')
print(len(filename))
filename=glob.glob('train/dog*')
print(len(filename))

1000
11500


In [13]:
if os.path.isdir('valid/cat') is False:
  os.makedirs('valid/cat')
  for c in random.sample(glob.glob('train/cat*'),1000):
    shutil.move(c,'valid/cat')

In [14]:
filename=glob.glob('valid/cat/*')
print(len(filename))
filename=glob.glob('train/cat*')
print(len(filename))

1000
11500


In [15]:
if os.path.isdir('test/cat') is False:
  os.makedirs('test/cat')
  for c in random.sample(glob.glob('train/cat*'),500):
    shutil.move(c,'test/cat')

In [16]:
filename=glob.glob('test/cat/*')
print(len(filename))
filename=glob.glob('train/cat*')
print(len(filename))

500
11000


In [17]:
if os.path.isdir('test/dog') is False:
  os.makedirs('test/dog')
  for c in random.sample(glob.glob('train/dog*'),500):
    shutil.move(c,'test/dog')

In [23]:
filename=glob.glob('test/dog/*')
print(len(filename))
filename=glob.glob('train/dog*')
print(len(filename))

500
2


In [19]:
if os.path.isdir('train/dog') is False:
  os.makedirs('train/dog')
  for c in random.sample(glob.glob('train/dog*'),11000):
    shutil.move(c,'train/dog')

In [20]:
if os.path.isdir('train/cat') is False:
  os.makedirs('train/cat')
  for c in random.sample(glob.glob('train/cat*'),11000):
    shutil.move(c,'train/cat')

In [22]:
filename=glob.glob('train/cat/*')
print("Cats in cat folder of Train ",len(filename))
filename=glob.glob('train/dog/*')
print("Dogs in dog folder of Train ",len(filename))

Cats in cat folder of Train  10999
Dogs in dog folder of Train  10999


Whenever we train a model, we need to put the data into a format that the model expects. A karass sequential model receives the data whenever we call the fit function. 
We create train, valid and test batches by using `ImageDataGenerator` and a directory iterator `flow_from_directory`. This basically creates batches of data from the directories we just created for training, validation and testing.

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

So let's focus just on train batches for now. So we're setting train batches equal to image data generator dot flow from directory.
But first to image data generator, we are specifying this pre processing function and setting that equal to tf Keras dot applications that VGG 16 dot pre process input.
This is a function that is going to apply some type of pre processing on the images before they get passed to the network that we'll be using. And we're processing them in such a way that is equivalent to the way that a very popular
model known as VGG 16, we're processing our images in the same format as which images that get passed to this VGG 16 model our process. And we're going to talk about more about this in a future episode.   

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

So besides that, when we call flow from directory, this is where we are passing in our actual data and specifying how we want this data to be processed. So we are setting directory equal to train path, which appear we defined the location on disk where our training set was under the train PATH variable. And then we're setting target size equal to 224 by 224.
So if you're working with an image data set that has images of varying sizes, or you just want to scale them up or scale them down, this is how you can specify that to happen. And this will resize all images in your data set to be of this height and width before passing them to our network. Now we are specifying our classes, which are just the classes for the potential labels of our data set. So cat or dog, and we are setting our batch size to 10.
We do the exact same thing for the validation set. And the test set.

Everything is the exact same for both of them, except for where each of these sets live on disk as being specified here under the directory parameter.
And then the only other difference is here for our test batches. We are specifying this shuffle equals false parameter.
Now, this is because whenever we use our test batches later for inference to get our model
to predict on images of cats and dogs after training and validation has been completed,
we're going to want to look at our prediction results in a confusion matrix like we did in a previous video for a separate data set.
And in order to do that, we need to be able to access the unsettled labels for our test
set. So that's why we set shuffle equals false for only this set. For both validation and training sets, we do want the data sets to be shuffled.